In [1]:
## Import libraries
## ---------------------------------------------------------

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import math
import xgboost as xgb
import xgboost as xgb
import warnings; warnings.simplefilter(action='ignore', category=FutureWarning)
import os

from sklearn.svm import SVR
from xgboost import XGBClassifier
from rdkit import Chem
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import RFE
from sklearn import svm
from sklearn import preprocessing
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn import ensemble
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from scipy import stats
from sklearn.feature_selection import f_classif,SelectKBest
from sklearn.decomposition import PCA
from sklearn.feature_selection import chi2
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import StackingClassifier
from sklearn.decomposition import PCA
from sklearn import model_selection
from sklearn import metrics
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import confusion_matrix
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import auc
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn import metrics
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import confusion_matrix
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn import linear_model

In [2]:
dataset_pad = pd.read_csv('PADEL.csv')
dataset_cdk = pd.read_csv('CDK.csv')
dataset_rdkit = pd.read_csv('RDKIT.csv')
dataset_mordred = pd.read_csv('MODERED.csv')
dataset_bluedesc = pd.read_csv('BLUDESC.csv')

In [3]:
dataset_pad = dataset_pad.iloc[:, 2:]#take data from dataframe row and coloumns
dataset_cdk = dataset_cdk.iloc[:, 2:]#take data from dataframe row and coloumns
dataset_rdkit = dataset_rdkit.iloc[:, 2:]#take data from dataframe row and coloumns
dataset_mordred = dataset_mordred.iloc[:, 2:]#take data from dataframe row and coloumns
dataset_bluedesc = dataset_mordred.iloc[:, 2:]#take data from dataframe row and coloumns

In [4]:
dataset_pad=dataset_pad.drop(["TARGET"],axis=1)
dataset_cdk=dataset_cdk.drop(["TARGET"],axis=1)
dataset_rdkit=dataset_rdkit.drop(["TARGET"],axis=1)
dataset_mordred=dataset_mordred.drop(["TARGET"],axis=1)

In [5]:
complete_data = pd.concat([dataset_pad, dataset_cdk, dataset_rdkit, dataset_mordred, dataset_bluedesc], axis=1)
complete_data =complete_data.dropna(axis=1)
complete_data.head()

,nAcid,ALogP,ALogp2,AMR,nA,nR,nN,nD,nC,nF,...,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2,TARGET
0,0,-0.2314,0.053546,44.7549,0,0,0,0,0,0,...,40.344003,151.063329,7.553166,166,11,50,53,4.583333,2.500000,1
1,0,-2.3401,5.476068,51.5213,0,0,0,0,0,0,...,62.224603,225.086189,8.336526,453,22,80,93,5.555556,3.666667,0
2,0,0.1145,0.013110,70.8731,0,0,0,0,0,0,...,49.239030,239.152143,6.293477,560,22,82,90,8.006944,3.763889,0
3,4,-2.4572,6.037832,42.8430,0,0,0,0,0,0,...,46.206502,249.016725,9.222842,281,23,68,78,8.937500,3.000000,0
4,0,-0.9351,0.874412,33.8177,0,0,0,0,0,0,...,53.745115,136.038511,9.717036,105,12,52,61,2.833333,2.222222,0


In [6]:
#VARIANCE THRESHOLD to REMOVE the CONSTANT FEATURES
from sklearn.feature_selection import VarianceThreshold
var_thres=VarianceThreshold(threshold=0)
var_thres.fit(complete_data)

var_thres.get_support()
complete_data.columns[var_thres.get_support()]

constant_columns = [column for column in complete_data.columns
                    if column not in complete_data.columns[var_thres.get_support()]]

print(len(constant_columns))

680


In [7]:
complete_data2=complete_data.drop(constant_columns,axis=1)


In [8]:
# Correlation
def correlation(dataset, threshold):
    col_corr = set() 
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold: 
                colname = corr_matrix.columns[i]  
                col_corr.add(colname)
    return col_corr

In [9]:
corr_features  = correlation(complete_data2, 0.95)
complete_data3 = complete_data2.drop(corr_features,axis=1)

In [10]:
#Min-Max Scaler
minmax_scale = MinMaxScaler()
minmax_scale.fit((complete_data3))
df_exp_minmax=minmax_scale.transform(complete_data3)
print(type(df_exp_minmax))
df_exp_minmax_new = pd.DataFrame(df_exp_minmax,columns=complete_data3.columns)
df_exp_minmax_new

<class 'numpy.ndarray'>


,ALogP,ALogp2,AMR,nA,nR,nD,nE,nL,nK,nM,...,fr_sulfonamd,fr_sulfone,fr_thiazole,fr_thiophene,fr_unbrch_alkane,fr_urea,ABC,n12Ring,n12Ring,TARGET
0,0.811380,0.000166,0.054328,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.118425,0.0,0.0,1.0
1,0.714805,0.016999,0.068298,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.2,0.0,0.177035,0.0,0.0,0.0
2,0.827222,0.000040,0.108253,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.184590,0.0,0.0,0.0
3,0.709442,0.018743,0.050380,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.147774,0.0,0.0,0.0
4,0.779152,0.002714,0.031746,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.114047,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220,0.813848,0.000097,0.122514,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.210319,0.0,0.0,0.0
221,0.845802,0.000840,0.222442,0.066667,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.355077,0.0,0.0,0.0
222,0.453091,0.201402,0.694635,0.466667,0.0,0.0,0.0,0.2,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,1.0
223,0.800659,0.000672,0.128526,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.220958,0.0,0.0,0.0


In [43]:
X = df_exp_minmax_new.iloc[:,0:-1]
y= df_exp_minmax_new["TARGET"]

from sklearn.feature_selection import SelectPercentile, chi2
X_new = SelectPercentile(chi2, percentile=10).fit_transform(X, y)
#y.shape
#print(y)

In [12]:
X_new.shape #(455,75)

(225, 45)

In [13]:
### train Test split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_new,y,test_size=0.2,random_state=42)

In [14]:
X_train=np.array(X_train)
X_test=np.array(X_test)
y_train=np.array(y_train)
y_test=np.array(y_test)

In [15]:
# cross validation 
def CF_Validation(k,train_data,train_targets):
    num_val_samples = len(train_data) // k   # number of samples is divided by number of folds
    total_validation_data=[]
    total_validation_targets=[]
    total_train_data=[]
    total_train_targets=[]
    
    for i in range(k):
        
        val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
        val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]
        
        partial_train_data = np.concatenate(
            [train_data[:i * num_val_samples],
            train_data[(i + 1) * num_val_samples:]],
            axis=0)
        partial_train_targets = np.concatenate(
            [train_targets[:i * num_val_samples],
            train_targets[(i + 1) * num_val_samples:]],
            axis=0)
    
        total_validation_data.append(val_data)
        total_validation_targets.append(val_targets) 
    
        total_train_data.append(partial_train_data)
        total_train_targets.append(partial_train_targets)
    return total_train_data,total_train_targets,total_validation_data,total_validation_targets

In [16]:
def get_vec_data(coll_vec,coll_label,k):
    vec_data =[]
    t_vec =[]
    for i in range(k):
        vec_data = coll_vec[i], coll_label[i]
        t_vec.append(vec_data)
    return t_vec

In [17]:
def Machine_Learning(k,ML_model,t_vec_train,t_vec_val,t_vec_test):
    total_y_pred_train =[]
    total_y_pred_test =[]
    total_y_true_train =[]
    total_y_true_test =[]
    
    for i in range(k):
        model = ML_model

        # data have 5 different arrangements (5 fold)
        # Train the model using the training sets
        features = t_vec_train[i][0]
        label = t_vec_train[i][1]
        model.fit(features,label)

        #Predict Output 
        # evaluation of training session
        features_v = t_vec_val[i][0]
        label_v  = t_vec_val[i][1]
        
        features_train = np.concatenate((features,features_v),axis=0)
        
        y_pred_train= model.predict(features_train)
        y_true_train =np.concatenate((label,label_v),axis=0) 
        
               
        #evaluation of test session
        
        y_pred_test= model.predict(t_vec_test[0])
        y_true_test = t_vec_test[1]
        
        total_y_pred_train.append(y_pred_train)
        total_y_pred_test.append(y_pred_test)
        total_y_true_train.append(y_true_train)
        total_y_true_test.append(y_true_test)
        
    return total_y_pred_train,total_y_true_train,total_y_pred_test,total_y_true_test

In [18]:
def Total_Performances(k, total_y_pred_train,total_y_true_train,total_y_pred_test,total_y_true_test):
    total_cm_train=[]
    total_cm_test=[]
    total_accuracy_train=[]
    total_accuracy_test=[]

    total_sensitivity_train=[]
    total_sensitivity_test=[]

    total_specificity_train=[]
    total_specificity_test=[]

    total_MCC_train =[]
    total_MCC_test =[]

    total_AUC_train =[]
    total_AUC_test =[]

    for j in range(k):
        y_pred_train = total_y_pred_train[j]
        y_pred_test = total_y_pred_test[j] 
        y_true_train =total_y_true_train[j]
        y_true_test = total_y_true_test[j]
        
        for i in range(len(y_pred_train)):
            if y_pred_train[i] <0.5 :
                y_pred_train[i] = 0
            else :
                y_pred_train[i] = 1
    
        for i in range(len(y_pred_test)):
            if y_pred_test[i] <0.5:
                y_pred_test[i] = 0
            else :
                y_pred_test[i] = 1   
            
        total_y_pred_train.append(y_pred_train)
        total_y_pred_test.append(y_pred_test)
    
        total_cm_train.append(confusion_matrix(y_true_train, y_pred_train))
        total_cm_test.append(confusion_matrix(y_true_test, y_pred_test))
    
    
        total_MCC_train.append(matthews_corrcoef(y_true_train, y_pred_train))
        total_MCC_test.append(matthews_corrcoef(y_true_test, y_pred_test))     
        fpr1, tpr1, thresholds1 = roc_curve(y_true_train, y_pred_train, pos_label=1)
        fpr2, tpr2, thresholds2 = roc_curve(y_true_test, y_pred_test, pos_label=1)
        total_AUC_train.append(auc(fpr1, tpr1))
        total_AUC_test.append(auc(fpr2,tpr2))
        
    for i in range(k):
        c_m_train = total_cm_train[i]
        c_m_test = total_cm_test[i]
        total_cmtrain=sum(sum(c_m_train))
        total_cmtest=sum(sum(c_m_test))
    
        trainAccuracy = (c_m_train[0,0]+c_m_train[1,1])/total_cmtrain
        testAccuracy = (c_m_test[0,0]+c_m_test[1,1])/total_cmtest
    
        trainSensitivity = c_m_train[1,1]/(c_m_train[1,0]+c_m_train[1,1])
        testSensitivity = c_m_test[1,1]/(c_m_test[1,0]+c_m_test[1,1])
    
        trainSpecificity = c_m_train[0,0]/(c_m_train[0,0]+c_m_train[0,1])
        testSpecificity = c_m_test[0,0]/(c_m_test[0,0]+c_m_test[0,1])               
    
        total_accuracy_train.append(trainAccuracy)
        total_accuracy_test.append(testAccuracy)
        total_sensitivity_train.append(trainSensitivity)
        total_sensitivity_test.append(testSensitivity)                      
        total_specificity_train.append(trainSpecificity)
        total_specificity_test.append(testSpecificity)
        
    for i in range (k): 
        total_accuracy_train[i]    =  round(total_accuracy_train[i],4)
        total_accuracy_test[i]     =  round(total_accuracy_test[i],4)
        total_sensitivity_train[i] =  round(total_sensitivity_train[i],4)
        total_sensitivity_test[i]  =  round(total_sensitivity_test[i],4)
        total_specificity_train[i] =  round(total_specificity_train[i],4)
        total_specificity_test[i]  =  round(total_specificity_test[i],4)
        total_MCC_train[i]         =  round(total_MCC_train[i],4)
        total_MCC_test[i]          =  round(total_MCC_test[i],4)
        total_AUC_train[i]         =  round(total_AUC_train[i],4)
        total_AUC_test[i]          =  round(total_AUC_test[i],4)     
        
        total_performances_train = [total_accuracy_train,total_sensitivity_train,total_specificity_train, total_MCC_train,total_AUC_train]
        total_performances_test = [total_accuracy_test,total_sensitivity_test,total_specificity_test, total_MCC_test,total_AUC_test]
    return total_performances_train, total_performances_test

In [19]:
###############TESTING ONE BY ONE #######################

In [20]:
def KNN(k,t_vec_train,t_vec_val,t_vec_test):
    #ML_model =KNeighborsClassifier(n_neighbors=3) first running
    ML_model =KNeighborsClassifier(n_neighbors=9)
    return Machine_Learning(k,ML_model,t_vec_train,t_vec_val,t_vec_test)

def XGB(k,t_vec_train,t_vec_val,t_vec_test):
    #ML_model =xgb.XGBClassifier(n_estimators=200, learning_rate=1,max_depth=1, random_state=42)
    ML_model =xgb.XGBClassifier(n_estimators=200, learning_rate=0.05,max_depth=1, random_state=42)
    return Machine_Learning(k,ML_model,t_vec_train,t_vec_val,t_vec_test)

def SVM(k,t_vec_train,t_vec_val,t_vec_test):
    #ML_model =svm.SVC(C=1, gamma= 'scale', kernel= 'rbf', random_state=42) first running
    ML_model =svm.SVC(C=5, gamma= 0.1, kernel= 'rbf', random_state=42)
    return Machine_Learning(k,ML_model,t_vec_train,t_vec_val,t_vec_test)

def LR(k,t_vec_train,t_vec_val,t_vec_test):
    #ML_model =LogisticRegression(solver='liblinear',random_state=42)first running
    ML_model =LogisticRegression(solver='liblinear',random_state=42,C=1)
    return Machine_Learning(k,ML_model,t_vec_train,t_vec_val,t_vec_test)

def RF(k,t_vec_train,t_vec_val,t_vec_test):
    #ML_model =RandomForestClassifier(n_estimators=500,random_state=42,n_jobs=-1)first running
    ML_model =RandomForestClassifier(n_estimators=5,random_state=42,n_jobs=-1,max_depth=25, max_leaf_nodes=5,min_samples_leaf=5)
    #max_depth=25, max_leaf_nodes=5,min_samples_leaf=5, n_estimators=5
    return Machine_Learning(k,ML_model,t_vec_train,t_vec_val,t_vec_test)

def Lasso(k,t_vec_train,t_vec_val,t_vec_test):
    ML_model =linear_model.Lasso(alpha=0.01)
    return Machine_Learning(k,ML_model,t_vec_train,t_vec_val,t_vec_test)

In [21]:
total_train_data,total_train_targets,total_validation_data,total_validation_targets = CF_Validation(k=5,train_data = X_train,train_targets=y_train)

In [22]:
t_vec_train =get_vec_data(total_train_data,total_train_targets,5)
t_vec_val = get_vec_data(total_validation_data,total_validation_targets,5)

t_vec_test= [X_test,y_test]

In [23]:
total_y_pred_train,total_y_true_train,total_y_pred_test,total_y_true_test=KNN(5,t_vec_train,t_vec_val,t_vec_test)
KNN_total_performances_train1,KNN_total_performances_test1 = Total_Performances(5, total_y_pred_train,total_y_true_train,
                                                                         total_y_pred_test,total_y_true_test)

In [24]:
KNN_total_performances_train1,KNN_total_performances_test1 #acc test score = 0.75556

([[0.6667, 0.7, 0.6722, 0.6722, 0.6722],
  [0.5, 0.5132, 0.4737, 0.4868, 0.4211],
  [0.7885, 0.8365, 0.8173, 0.8077, 0.8558],
  [0.3022, 0.3731, 0.312, 0.3127, 0.3113],
  [0.6442, 0.6748, 0.6455, 0.6473, 0.6384]],
 [[0.7556, 0.7111, 0.7556, 0.8, 0.8],
  [0.5625, 0.375, 0.5, 0.5625, 0.5],
  [0.8621, 0.8966, 0.8966, 0.931, 0.9655],
  [0.4484, 0.325, 0.4417, 0.5497, 0.5571],
  [0.7123, 0.6358, 0.6983, 0.7468, 0.7328]])

In [25]:
total_y_pred_train,total_y_true_train,total_y_pred_test,total_y_true_test=XGB(5,t_vec_train,t_vec_val,t_vec_test)
XGB_total_performances_train1,XGB_total_performances_test1 = Total_Performances(5, total_y_pred_train,total_y_true_train,
                                                                         total_y_pred_test,total_y_true_test)

In [26]:
XGB_total_performances_train1,XGB_total_performances_test1 #acc test score =0.6266

([[0.6944, 0.7389, 0.7111, 0.75, 0.7056],
  [0.4605, 0.5658, 0.4474, 0.5921, 0.4211],
  [0.8654, 0.8654, 0.9038, 0.8654, 0.9135],
  [0.3617, 0.4578, 0.4036, 0.4814, 0.3939],
  [0.663, 0.7156, 0.6756, 0.7287, 0.6673]],
 [[0.6444, 0.6222, 0.6889, 0.5778, 0.6444],
  [0.25, 0.1875, 0.1875, 0.25, 0.125],
  [0.8621, 0.8621, 0.9655, 0.7586, 0.931],
  [0.1403, 0.0655, 0.2574, 0.0096, 0.0943],
  [0.556, 0.5248, 0.5765, 0.5043, 0.528]])

In [27]:
total_y_pred_train,total_y_true_train,total_y_pred_test,total_y_true_test=SVM(5,t_vec_train,t_vec_val,t_vec_test)
SVM_total_performances_train1,SVM_total_performances_test1 = Total_Performances(5, total_y_pred_train,total_y_true_train,
                                                                         total_y_pred_test,total_y_true_test)

In [28]:
SVM_total_performances_train1,SVM_total_performances_test1 #acc test score =0.75112

([[0.7444, 0.7722, 0.7556, 0.7333, 0.7333],
  [0.5132, 0.5263, 0.4342, 0.4474, 0.4211],
  [0.9135, 0.9519, 0.9904, 0.9423, 0.9615],
  [0.4765, 0.5455, 0.5358, 0.4629, 0.4724],
  [0.7133, 0.7391, 0.7123, 0.6948, 0.6913]],
 [[0.8222, 0.7556, 0.7333, 0.7556, 0.7556],
  [0.5, 0.375, 0.3125, 0.375, 0.3125],
  [1.0, 0.9655, 0.9655, 0.9655, 1.0],
  [0.626, 0.4497, 0.3915, 0.4497, 0.476],
  [0.75, 0.6703, 0.639, 0.6703, 0.6562]])

In [29]:
total_y_pred_train,total_y_true_train,total_y_pred_test,total_y_true_test=LR(5,t_vec_train,t_vec_val,t_vec_test)
LR_total_performances_train1,LR_total_performances_test1 = Total_Performances(5, total_y_pred_train,total_y_true_train,
                                                                         total_y_pred_test,total_y_true_test)

In [30]:
LR_total_performances_train1,LR_total_performances_test1 #acc test score =0.77334

([[0.7111, 0.7222, 0.6944, 0.6944, 0.7],
  [0.5395, 0.5263, 0.4342, 0.4342, 0.4079],
  [0.8365, 0.8654, 0.8846, 0.8846, 0.9135],
  [0.3974, 0.4222, 0.3637, 0.3637, 0.3818],
  [0.688, 0.6959, 0.6594, 0.6594, 0.6607]],
 [[0.8222, 0.7556, 0.7556, 0.8, 0.7333],
  [0.5625, 0.375, 0.375, 0.5, 0.3125],
  [0.9655, 0.9655, 0.9655, 0.9655, 0.9655],
  [0.608, 0.4497, 0.4497, 0.5571, 0.3915],
  [0.764, 0.6703, 0.6703, 0.7328, 0.639]])

In [31]:
total_y_pred_train,total_y_true_train,total_y_pred_test,total_y_true_test=RF(5,t_vec_train,t_vec_val,t_vec_test)
RF_total_performances_train1,RF_total_performances_test1 = Total_Performances(5, total_y_pred_train,total_y_true_train,
                                                                         total_y_pred_test,total_y_true_test)

In [32]:
RF_total_performances_train1,RF_total_performances_test1 #acc test score =0.68

([[0.6611, 0.6278, 0.6611, 0.6389, 0.6278],
  [0.3816, 0.4474, 0.3289, 0.3816, 0.3421],
  [0.8654, 0.7596, 0.9038, 0.8269, 0.8365],
  [0.2861, 0.2178, 0.2905, 0.2345, 0.2069],
  [0.6235, 0.6035, 0.6164, 0.6043, 0.5893]],
 [[0.7333, 0.6, 0.6889, 0.6222, 0.6444],
  [0.3125, 0.25, 0.25, 0.125, 0.125],
  [0.9655, 0.7931, 0.931, 0.8966, 0.931],
  [0.3915, 0.0496, 0.2549, 0.0328, 0.0943],
  [0.639, 0.5216, 0.5905, 0.5108, 0.528]])

In [33]:
total_y_pred_train,total_y_true_train,total_y_pred_test,total_y_true_test=Lasso(5,t_vec_train,t_vec_val,t_vec_test)
Lasso_total_performances_train1,Lasso_total_performances_test1 = Total_Performances(5, total_y_pred_train,total_y_true_train,
                                                                         total_y_pred_test,total_y_true_test)

In [34]:
Lasso_total_performances_train1,Lasso_total_performances_test1#acc test score =0.6844

([[0.65, 0.65, 0.6611, 0.6278, 0.6556],
  [0.3816, 0.3026, 0.3026, 0.2368, 0.2632],
  [0.8462, 0.9038, 0.9231, 0.9135, 0.9423],
  [0.2598, 0.2636, 0.2953, 0.2079, 0.2887],
  [0.6139, 0.6032, 0.6129, 0.5752, 0.6027]],
 [[0.7556, 0.6667, 0.6667, 0.6667, 0.6667],
  [0.4375, 0.125, 0.125, 0.125, 0.0625],
  [0.931, 0.9655, 0.9655, 0.9655, 1.0],
  [0.441, 0.1737, 0.1737, 0.1737, 0.203],
  [0.6843, 0.5453, 0.5453, 0.5453, 0.5312]])

In [35]:
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA

In [36]:
pipeline_randomforest=Pipeline([('scalar1',StandardScaler()),
                     ('pca1',PCA()),
                     ('rf_classifier',RandomForestClassifier(random_state=100))])
pipeline_svm=Pipeline([('scalar2',StandardScaler()),
                     ('pca2',PCA()),
                     ('svm_classifier',svm.SVC())])
pipeline_lr=Pipeline([('scalar3',StandardScaler()),
                     ('pca3',PCA()),
                     ('lr_classifier',LogisticRegression(random_state=0))])
pipeline_xgb=Pipeline([('scalar5',StandardScaler()),
                       ('pca5',PCA()),
                       ("clf", xgb.XGBClassifier())])
pipeline_knn=Pipeline([('scalar6',StandardScaler()),
                       ('pca6',PCA()),
                       ("knn_classifier", KNeighborsClassifier())])

In [37]:
pipelines = [pipeline_randomforest, pipeline_svm, pipeline_lr,pipeline_xgb,pipeline_knn]

In [38]:
best_accuracy=0.0
best_classifier=0
best_pipeline=""

In [39]:
# Dictionary of pipelines and classifier types for ease of reference
pipe_dict = {0: 'RandomForest', 1: 'svm', 2: 'LogisticRegression', 3:'xgb.XGBClassifier',
             4: 'KNeighborsClassifier'}

# Fit the pipelines
for pipe in pipelines:
    pipe.fit(X_train, y_train)

In [40]:
for i,model in enumerate(pipelines):
    print("{} Test Accuracy: {}".format(pipe_dict[i],model.score(X_test, y_test)))

RandomForest Test Accuracy: 0.7333333333333333
svm Test Accuracy: 0.7555555555555555
LogisticRegression Test Accuracy: 0.7777777777777778
xgb.XGBClassifier Test Accuracy: 0.7555555555555555
KNeighborsClassifier Test Accuracy: 0.8


In [44]:
# Create a pipeline
pipe = Pipeline([("classifier", RandomForestClassifier())])
# Create dictionary with candidate learning algorithms and their hyperparameters
grid_param = [
                 {"classifier": [RandomForestClassifier()],
                 "classifier__n_estimators": [5,10,40,60,80,100,120,400],
                 "classifier__max_depth":[5,8,15,25,30,None],
                 "classifier__min_samples_leaf":[1,2,5,10,15,100],
                 "classifier__max_leaf_nodes": [2, 5,10]}]
# create a gridsearch of the pipeline, the fit the best model
gridsearch = GridSearchCV(pipe, grid_param, cv=10, verbose=0,n_jobs=-1) # Fit grid search
best_model = gridsearch.fit(X_train,y_train)

In [45]:
#RANDOM FOREST
print(best_model.best_estimator_)
print("The mean accuracy of the model is:",best_model.score(X_test,y_test))

Pipeline(steps=[('classifier',
                 RandomForestClassifier(max_depth=30, max_leaf_nodes=5,
                                        min_samples_leaf=2, n_estimators=40))])
The mean accuracy of the model is: 0.6888888888888889


In [46]:
## GRID SEARCH SVM
pipe = Pipeline([("classifier", SVC())])
# Create dictionary with candidate learning algorithms and their hyperparameters
grid_param = [
                 {"classifier":[SVC()],
                 "classifier__kernel":['rbf'],
                 "classifier__C":[0.01,0.1,1,5,10,15,20,50,100],
                 "classifier__gamma":[0.0001,0.001,0.01,0.1,1,5,10,15,20,50,100]}]
# create a gridsearch of the pipeline, the fit the best model
gridsearch = GridSearchCV(pipe, grid_param, cv=3, verbose=0,n_jobs=-1) # Fit grid search
best_model = gridsearch.fit(X_train,y_train)

In [47]:
print(best_model.best_estimator_)
print("The mean accuracy of the model is:",best_model.score(X_test,y_test))

Pipeline(steps=[('classifier', SVC(C=5, gamma=0.1))])
The mean accuracy of the model is: 0.7333333333333333


In [48]:
##Pipeline LR
pipe = Pipeline([("classifier", LogisticRegression())])
# Create dictionary with candidate learning algorithms and their hyperparameters
grid_param = [
                 {"classifier": [LogisticRegression()],
                 "classifier__C": [1,5,10,50,100,200,500],
                 "classifier__multi_class":['auto'],
                 "classifier__solver":['liblinear']}]
# create a gridsearch of the pipeline, the fit the best model
gridsearch = GridSearchCV(pipe, grid_param, cv=10, verbose=0,n_jobs=-1) # Fit grid search
best_model = gridsearch.fit(X_train,y_train)

In [49]:
print(best_model.best_estimator_)
print("The mean accuracy of the model is:",best_model.score(X_test,y_test))

Pipeline(steps=[('classifier', LogisticRegression(C=1, solver='liblinear'))])
The mean accuracy of the model is: 0.8


In [50]:
##Pipeline KNN
pipe = Pipeline([("classifier", KNeighborsClassifier())])
# Create dictionary with candidate learning algorithms and their hyperparameters
grid_param = [
                 {"classifier": [KNeighborsClassifier()],
                 "classifier__n_neighbors": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]}]
# create a gridsearch of the pipeline, the fit the best model
gridsearch = GridSearchCV(pipe, grid_param, cv=10, verbose=0,n_jobs=-1) # Fit grid search
best_model = gridsearch.fit(X_train,y_train)

In [51]:
print(best_model.best_estimator_)
print("The mean accuracy of the model is:",best_model.score(X_test,y_test))

Pipeline(steps=[('classifier', KNeighborsClassifier(n_neighbors=9))])
The mean accuracy of the model is: 0.7333333333333333


In [52]:
#######################AFTER GRID SEARCH CV############################

In [53]:
def KNN(k,t_vec_train,t_vec_val,t_vec_test):
    #ML_model =KNeighborsClassifier(n_neighbors=3) first running
    ML_model =KNeighborsClassifier(n_neighbors=9)
    return Machine_Learning(k,ML_model,t_vec_train,t_vec_val,t_vec_test)

def SVM(k,t_vec_train,t_vec_val,t_vec_test):
    #ML_model =svm.SVC(C=1, gamma= 'scale', kernel= 'rbf', random_state=42) first running
    ML_model =svm.SVC(C=5, gamma= 0.1, kernel= 'rbf', random_state=42)
    return Machine_Learning(k,ML_model,t_vec_train,t_vec_val,t_vec_test)

def LR(k,t_vec_train,t_vec_val,t_vec_test):
    #ML_model =LogisticRegression(solver='liblinear',random_state=42)first running
    ML_model =LogisticRegression(solver='liblinear',random_state=42,C=1)
    return Machine_Learning(k,ML_model,t_vec_train,t_vec_val,t_vec_test)

def RF(k,t_vec_train,t_vec_val,t_vec_test):
    #ML_model =RandomForestClassifier(n_estimators=500,random_state=42,n_jobs=-1)first running
    ML_model =RandomForestClassifier(n_estimators=5,random_state=42,n_jobs=-1,max_depth=25, max_leaf_nodes=5,min_samples_leaf=5)
    #max_depth=25, max_leaf_nodes=5,min_samples_leaf=5, n_estimators=5
    return Machine_Learning(k,ML_model,t_vec_train,t_vec_val,t_vec_test)

In [54]:
total_y_pred_train,total_y_true_train,total_y_pred_test,total_y_true_test=KNN(5,t_vec_train,t_vec_val,t_vec_test)
KNN_total_performances_train1,KNN_total_performances_test1 = Total_Performances(5, total_y_pred_train,total_y_true_train,
                                                                         total_y_pred_test,total_y_true_test)

In [55]:
KNN_total_performances_train1,KNN_total_performances_test1 #acc test score = 0.7644

([[0.6667, 0.7, 0.6722, 0.6722, 0.6722],
  [0.5, 0.5132, 0.4737, 0.4868, 0.4211],
  [0.7885, 0.8365, 0.8173, 0.8077, 0.8558],
  [0.3022, 0.3731, 0.312, 0.3127, 0.3113],
  [0.6442, 0.6748, 0.6455, 0.6473, 0.6384]],
 [[0.7556, 0.7111, 0.7556, 0.8, 0.8],
  [0.5625, 0.375, 0.5, 0.5625, 0.5],
  [0.8621, 0.8966, 0.8966, 0.931, 0.9655],
  [0.4484, 0.325, 0.4417, 0.5497, 0.5571],
  [0.7123, 0.6358, 0.6983, 0.7468, 0.7328]])

In [56]:
total_y_pred_train,total_y_true_train,total_y_pred_test,total_y_true_test=SVM(5,t_vec_train,t_vec_val,t_vec_test)
SVM_total_performances_train1,SVM_total_performances_test1 = Total_Performances(5, total_y_pred_train,total_y_true_train,
                                                                         total_y_pred_test,total_y_true_test)

In [57]:
SVM_total_performances_train1,SVM_total_performances_test1 #acc test score =0.7644

([[0.7444, 0.7722, 0.7556, 0.7333, 0.7333],
  [0.5132, 0.5263, 0.4342, 0.4474, 0.4211],
  [0.9135, 0.9519, 0.9904, 0.9423, 0.9615],
  [0.4765, 0.5455, 0.5358, 0.4629, 0.4724],
  [0.7133, 0.7391, 0.7123, 0.6948, 0.6913]],
 [[0.8222, 0.7556, 0.7333, 0.7556, 0.7556],
  [0.5, 0.375, 0.3125, 0.375, 0.3125],
  [1.0, 0.9655, 0.9655, 0.9655, 1.0],
  [0.626, 0.4497, 0.3915, 0.4497, 0.476],
  [0.75, 0.6703, 0.639, 0.6703, 0.6562]])

In [58]:
total_y_pred_train,total_y_true_train,total_y_pred_test,total_y_true_test=LR(5,t_vec_train,t_vec_val,t_vec_test)
LR_total_performances_train1,LR_total_performances_test1 = Total_Performances(5, total_y_pred_train,total_y_true_train,
                                                                         total_y_pred_test,total_y_true_test)

In [59]:
LR_total_performances_train1,LR_total_performances_test1 #acc test score =0.7733

([[0.7111, 0.7222, 0.6944, 0.6944, 0.7],
  [0.5395, 0.5263, 0.4342, 0.4342, 0.4079],
  [0.8365, 0.8654, 0.8846, 0.8846, 0.9135],
  [0.3974, 0.4222, 0.3637, 0.3637, 0.3818],
  [0.688, 0.6959, 0.6594, 0.6594, 0.6607]],
 [[0.8222, 0.7556, 0.7556, 0.8, 0.7333],
  [0.5625, 0.375, 0.375, 0.5, 0.3125],
  [0.9655, 0.9655, 0.9655, 0.9655, 0.9655],
  [0.608, 0.4497, 0.4497, 0.5571, 0.3915],
  [0.764, 0.6703, 0.6703, 0.7328, 0.639]])

In [60]:
total_y_pred_train,total_y_true_train,total_y_pred_test,total_y_true_test=RF(5,t_vec_train,t_vec_val,t_vec_test)
RF_total_performances_train1,RF_total_performances_test1 = Total_Performances(5, total_y_pred_train,total_y_true_train,
                                                                         total_y_pred_test,total_y_true_test)

In [61]:
RF_total_performances_train1,RF_total_performances_test1 #acc test score =0.6577

([[0.6611, 0.6278, 0.6611, 0.6389, 0.6278],
  [0.3816, 0.4474, 0.3289, 0.3816, 0.3421],
  [0.8654, 0.7596, 0.9038, 0.8269, 0.8365],
  [0.2861, 0.2178, 0.2905, 0.2345, 0.2069],
  [0.6235, 0.6035, 0.6164, 0.6043, 0.5893]],
 [[0.7333, 0.6, 0.6889, 0.6222, 0.6444],
  [0.3125, 0.25, 0.25, 0.125, 0.125],
  [0.9655, 0.7931, 0.931, 0.8966, 0.931],
  [0.3915, 0.0496, 0.2549, 0.0328, 0.0943],
  [0.639, 0.5216, 0.5905, 0.5108, 0.528]])